<a href="https://colab.research.google.com/github/Ashritha0848/NLP/blob/main/2403A52229%2CAssignment_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import nltk
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import pandas as pd

df = pd.read_excel("/content/LDA-Data.xlsx")
corpus = df["News"].astype(str).tolist()

In [4]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

processed_corpus = [preprocess(doc) for doc in corpus]


In [5]:
count_vectorizer = CountVectorizer()
bow = count_vectorizer.fit_transform(processed_corpus)


In [6]:
bow_df = pd.DataFrame(
    bow.toarray(),
    columns=count_vectorizer.get_feature_names_out()
)

print(bow_df.head())


   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [7]:
lda_model = LatentDirichletAllocation(
    n_components=2,
    random_state=42
)

lda_model.fit(bow)


LatentDirichletAllocation(n_components=2, random_state=42)

In [8]:
def display_topics(model, feature_names, num_words=5):
    for idx, topic in enumerate(model.components_):
        print(f"\nTopic {idx+1}:")
        print([feature_names[i] for i in topic.argsort()[-num_words:]])

display_topics(lda_model, count_vectorizer.get_feature_names_out())



Topic 1:
['election', 'state', 'congress', 'government', 'form']

Topic 2:
['took', 'scored', 'century', 'virat', 'match']


In [9]:
lda_topics = lda_model.transform(bow)
lda_topic_df = pd.DataFrame(lda_topics, columns=["Topic 1", "Topic 2"])
print(lda_topic_df)


    Topic 1   Topic 2
0  0.104491  0.895509
1  0.827120  0.172880
2  0.104491  0.895509
3  0.896349  0.103651


In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(processed_corpus)


In [11]:
nmf_model = NMF(
    n_components=2,
    random_state=42
)

nmf_model.fit(tfidf)


NMF(n_components=2, random_state=42)

In [12]:
display_topics(nmf_model, tfidf_vectorizer.get_feature_names_out())



Topic 1:
['virat', 'took', 'wicket', 'bumra', 'match']

Topic 2:
['state', 'government', 'form', 'bjp', 'election']


In [13]:
nmf_topics = nmf_model.transform(tfidf)
nmf_topic_df = pd.DataFrame(nmf_topics, columns=["Topic 1", "Topic 2"])
print(nmf_topic_df)


    Topic 1   Topic 2
0  0.579747  0.000000
1  0.000000  0.783491
2  0.579747  0.000000
3  0.000000  0.391745


KAGGLE DATASET


In [16]:
import re
import nltk
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
import pandas as pd

df = pd.read_csv(
    "/content/arxiv_data.csv",
    engine="python",
    on_bad_lines="skip"
)

print(df.head())
print(df.columns)


                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  
0           ['cs.CV', 'cs.LG']  
1  ['cs.CV', 'cs.AI', 'cs.LG']  
2           ['cs.CV', 'cs.AI']  
3                    ['cs.CV']  
4           ['cs.CV', 'cs.LG']  
Index(['titles', 'summaries', 'terms'], dtype='object')


In [20]:
corpus = df["summaries"].astype(str).tolist()

In [21]:
print(len(corpus))
print(corpus[0])


20815
Stereo matching is one of the widely used techniques for inferring depth from
stereo images owing to its robustness and speed. It has become one of the major
topics of research since it finds its applications in autonomous driving,
robotic navigation, 3D reconstruction, and many other fields. Finding pixel
correspondences in non-textured, occluded and reflective areas is the major
challenge in stereo matching. Recent developments have shown that semantic cues
from image segmentation can be used to improve the results of stereo matching.
Many deep neural network architectures have been proposed to leverage the
advantages of semantic segmentation in stereo matching. This paper aims to give
a comparison among the state of art networks both in terms of accuracy and in
terms of speed which are of higher importance in real-time applications.


In [22]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

processed_corpus = [preprocess(doc) for doc in corpus]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    max_features=5000
)

bow = count_vectorizer.fit_transform(processed_corpus)


In [24]:
bow_df = pd.DataFrame(
    bow.toarray(),
    columns=count_vectorizer.get_feature_names_out()
)

bow_df.head()


,abdominal,ability,ablation,able,abnormal,abnormality,absence,absent,absolute,abstract,...,yield,yielded,yielding,yolo,yolov,youtube,zero,zone,zoom,zsl
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(
    n_components=5,
    random_state=42
)

lda_model.fit(bow)


LatentDirichletAllocation(n_components=5, random_state=42)

In [29]:
def display_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print([feature_names[i] for i in topic.argsort()[-num_words:]])

display_topics(lda_model, count_vectorizer.get_feature_names_out())



Topic 1:
['time', 'based', 'algorithm', 'method', 'neural', 'deep', 'data', 'network', 'learning', 'model']

Topic 2:
['using', 'model', 'face', 'result', 'proposed', 'based', 'network', 'method', 'segmentation', 'image']

Topic 3:
['generative', 'image', 'network', 'method', 'adversarial', 'domain', 'training', 'learning', 'model', 'data']

Topic 4:
['detector', 'dataset', 'performance', 'based', 'network', 'method', 'image', 'feature', 'detection', 'object']

Topic 5:
['network', 'method', 'attention', 'transformer', 'video', 'task', 'learning', 'model', 'graph', 'representation']


In [30]:
lda_output = lda_model.transform(bow)

lda_topic_df = pd.DataFrame(
    lda_output,
    columns=[f"Topic {i+1}" for i in range(5)]
)

lda_topic_df.head()


,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,0.239826,0.133178,0.002518,0.621960,0.002518
1,0.715411,0.279972,0.001541,0.001536,0.001540
2,0.001251,0.474855,0.521379,0.001257,0.001258
3,0.001570,0.143708,0.612551,0.191039,0.051132
4,0.339503,0.227127,0.001249,0.430876,0.001244


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000
)

tfidf = tfidf_vectorizer.fit_transform(processed_corpus)


In [32]:
from sklearn.decomposition import NMF

nmf_model = NMF(
    n_components=5,
    random_state=42
)

nmf_model.fit(tfidf)


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF(n_components=5, random_state=42)

In [33]:
display_topics(nmf_model, tfidf_vectorizer.get_feature_names_out())



Topic 1:
['adversarial', 'generative', 'deep', 'task', 'representation', 'training', 'network', 'data', 'learning', 'model']

Topic 2:
['frame', 'method', 'scene', 'box', 'point', 'feature', 'video', 'detector', 'detection', 'object']

Topic 3:
['embedding', 'network', 'information', 'structure', 'edge', 'gnns', 'learning', 'representation', 'node', 'graph']

Topic 4:
['proposed', 'based', 'region', 'pixel', 'network', 'semantic', 'method', 'medical', 'segmentation', 'image']

Topic 5:
['task', 'label', 'supervised', 'data', 'learning', 'transfer', 'adaptation', 'source', 'target', 'domain']


In [34]:
nmf_output = nmf_model.transform(tfidf)

nmf_topic_df = pd.DataFrame(
    nmf_output,
    columns=[f"Topic {i+1}" for i in range(5)]
)

nmf_topic_df.head()


,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,0.011056,0.011014,0.000000,0.037160,0.000000
1,0.007916,0.000000,0.000000,0.026165,0.001277
2,0.021621,0.001122,0.002075,0.059397,0.019309
3,0.029599,0.007970,0.002886,0.036252,0.014010
4,0.018270,0.021582,0.000000,0.028960,0.006489
